# Tâche 4 : Question-réponse avec affinage par instructions du modèle GPT‑2 pré-entraîné sur Sherlock Holmes

**Objectifs**

Évaluer la qualité des réponses d’un modèle de langage **pré‑entraîné** (celui de la tâche 3) et affiner par instructions (cette tâche).
Dans ce *notebook*, vous faites le post-entraînement du modèle avec des instructions générales indiquant au modèle comment accomplir des tâches simples. Comme pour les tâches 2 et 3, la démarche de test est de construire un *prompt*, de générer des réponses, et d'évaluer qualitativement la pertinence des résultats. Plusieurs de ces fonctions sont rendues disponibles. 

**Objectifs d’apprentissage**
1. Faire le post-entraînement d'un modèle pré‑entraîné avec l'affinage par instructions (*instruction tuning*).
2. Comprendre et expliquer les **limites et apports de l'affinage par instructions** d'un modèle.

Tout comme dans les tâches 2 et 3, les **questions** pour évaluer le modèle vous sont fournies. Le fichier d'**instructions** pour l'affinage du modèle est également fourni. Vous devez comprendre le format des questions chargées en mémoire. Il est également important de prendre connaissance de la nature des instructions utilisées pour l'affinage. 

> Il est recommandé de faire ce travail pratique en utilisant une carte graphique GPU compatible avec HuggingFace/Pytorch.
> Si votre machine n’en possède pas, vous pouvez utiliser **Google Colab** pour exécuter le *notebook* dans le cloud.

Si nécessaire, installer les *packages* suivant. Si vous exécutez sur Code Colab, ces *packages* devraient déjà être installés.

In [ ]:
#!pip install datasets
#!pip install accelerate
#!pip install 'transformers[torch]'
#!pip3 install torch torchvision

In [ ]:
batch_size = 5 # il est possible d'ajuster la taille de batch. Les valeurs actuelles utilisent environ 10 Gb
max_length = 256 # on réduit le contexte pour sauver du temps, nos exemples ne nécesside pas un plus grand contexte

In [ ]:
from datasets import Dataset
from transformers import pipeline, Trainer
import json

## 1. Chargement du modèle Hugging Face et du tokenizer (à compléter)

Complétez le corps de la fonction `load_model(model_path)` afin qu’elle :

- charge le **tokenizer** et le **modèle** Hugging Face à partir du chemin `model_path`.
- **retourne** le tokenizer comme **première valeur de retour** et le modèle comme **seconde valeur de retour**.

On ajoute également des fonctions pour monter les questions en mémoire et pour sauvegarder les réponses dans un fichier. 

In [ ]:
def load_model(model_path):
    tokenizer = None  # TODO
    model = None  # TODO
    
    return tokenizer, model

In [ ]:
def load_entries(path):
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    if not isinstance(data, list):
        raise ValueError(f"Question file must contain a list of objects. Got: {type(data)}")
    return data

def save_answers(questions_answers, output_dir, out_file_name, display=True):
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, out_file_name), "w", encoding="utf-8") as out:
        for index, question, answer, expected_answer in questions_answers:
            out.write(f"Q: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}\n")
            if display:
                print(f"Q{index}: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}")

## 2. Fonctions de test question-réponse  (à compléter)

La fonction **test_on_questions** est utilisée pour parcourir **toutes les entrées** du fichier de questions afin de produire des réponses générées par le modèle.

La génération d'une réponse à une question implique les étapes suivantes (fonction **process_entry** à compléter) : 
* Construire un prompt à l’aide de la fonction **alpaca_build_prompt** (rendu disponible dans la prochaine section)
* Utiliser le modèle (via un pipeline de génération de texte) pour générer une réponse à une question
* Retourner la réponse générée par le modèle.  

Points importants à souligner: 
* La fonction *process_entry* doit retourner uniquement la réponse générée par le modèle (sans le prompt).
* Il est de votre responsabililté de choisir **les paramètres** du générateur (max_new_tokens, do_sample, temperature, top_k ou top_p). Décrivez ceux que vous avez retenus.

> Afin de simplifier le travail, nous avons choisi de ne pas utiliser de *batchs* dans la fonction qui teste les questions.
> Vous n'avez pas à prendre en compte le *warning* qui suggère d'utiliser des *datasets*.

Description des paramètres de génération: 
(à compléter...)

In [ ]:
def process_entry(entry, prompt_builder, generator):
    answer = None  # TODO
    return answer

In [ ]:
def test_on_questions(prompt_builder, model_path, question_file, out_file_name, output_dir="results"):
    entries = load_entries(question_file)
    tokenizer, model = load_model(model_path)
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    results = []
    for i, entry in enumerate(entries):
        answer = process_entry({"instruction": entry["question"]}, prompt_builder, generator)
        question = entry.get("question", "")
        expected_answer = entry.get("answer", "")
        results.append((i, question, answer, expected_answer))
    save_answers(results, output_dir, out_file_name, display=True)
    return results

## 3. Préparation des données et des prompts pour l'affinage du modèle

Le code suivant prépare les ressources nécessaires pour l'affinage du modèle GPT2 pré-entraîné dans la tâche 3 de ce travail.

Les étapes sont :
* Télécharger le fichier de données Alpaca, le jeu d'instructions utilisé pour l'affinage du modèle . Afin de limiter le temps d'entraînement, on retient seulement les 5000 premières instructions de ce *dataset*. Vous pouvez modifier ce nombre si vous le souhaitez. 
* Générer un prompt spécifique à Alpaca.

On rend disponible tout ce qui est nécessaire pour ces 2 étapes. 

In [ ]:
import os
import urllib.request

alpaca_url = "https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/refs/heads/main/alpaca_data.json"

def load_or_download_instruct_dataset_file(data_url, file_path, count=-1):
    with urllib.request.urlopen(data_url) as response:
        raw_data = response.read().decode("utf-8")
        data = json.loads(raw_data)
    if count > 0 and count <= len(data):
        data = data[:count]
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

instructions_fn = "data/alpaca_data.json"  # Fichier où sont enregistrées les instructions d'affinage du modèle
nb_instructions = 5000  # Ce nombre peut-être modifié
load_or_download_instruct_dataset_file(data_url=alpaca_url, count=nb_instructions, file_path=instructions_fn)

In [ ]:
def alpaca_build_prompt(ex):
    instruction = ex.get("instruction", "")
    input = ex.get("input", "").strip()
    header = "Below is an instruction that describes a task"
    if input:
        header += ", paired with an input"
    return (
        f"{header}.\n"
        "Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{instruction}\n\n"
        + (f"### Input:\n{input}\n\n" if input else "")
        + "### Response:\n"
    )

## 4. Affinage du modèle (à compléter)

Complétez le code suivant pour affiner le modèle GPT2 préentraîné et sauvegardé dans la tâche 3 de ce travail.

Les étapes à suivre sont de :
* Monter en mémoire le modèle pré-entraîné à la tâche 3 et son tokeniseur 
* Monter le jeu d'instructions pour l'affinage du modèle et créer un *dataset* avec ces données.
* Tokeniser ce *dataset* d'instructions
* Faire l'entraînement du modèle sur le *dataset* avec la classe ***Trainer*** de Hugging Face
* Faire la sauvegarde du nouveau modèle dans un répertoire (voir *model_path*)

In [ ]:
model_name = "gpt2-sherlock-lm" # Répertoire du modèle construit durant la tâche 3
tokenizer, model = load_model(model_name)

Création du *dataset* d'entraînement.

In [ ]:
instructions_fn = "data/alpaca_data.json"  # Fichier qui contient les instructions d'affinage
dataset = None  # TODO

Tokénisation du *dataset* d'entraînement. 

In [ ]:
tokenized_dataset = None  # TODO

Ajouter dans ces cellules tout le code dont vous avez besoin pour faire l'affinage du modèle. 

In [ ]:
instruct_model_path = "gpt2-sherlock-instruct" # Répertoire où sauvegarder le nouveau modèle et le tokenizer 

trainer = None  # TODO

In [ ]:
trainer.train()

Pour conclure cette section, sauvegardez le nouveau modèle et le tokenizer. 

In [ ]:
trainer.save_model(instruct_model_path)  
tokenizer.save_pretrained(instruct_model_path)

## 5. Génération des réponses pour les questions de Sherlock Holmes

Exécutez la cellule suivante pour générer, avec le modèle affiné, les réponses aux questions de test.
Le temps d’exécution devrait se situer entre **5 et 10 minutes** si vous utilisez **Google Colab** avec un GPU.

Note : N'oubliez pas d'ajouter le fichier de réponses générées par le modèle (voir *out_file_name*) dans votre remise du travail.

In [ ]:
questions_fn = "data/questions_sherlock.json"
out_file_name = "instruct_gpt2_answers.txt"

results = test_on_questions(prompt_builder=alpaca_build_prompt, model_path=instruct_model_path, question_file=questions_fn, out_file_name=out_file_name)

 ## 6. Analyse des résultats 
 
 ### 6.1 Évaluation quantitative (à compléter)

In [ ]:
import string
import re
from collections import Counter

def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)

def white_space_fix(text):
    return ' '.join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()

def normalize_answer(s):
    """Mettre en minuscule et retirer la ponctuation, des déterminants and les espaces."""
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
def evaluate_f1(ground_truth, prediction):
    """Normalise les 2 textes, trouve ce qu'il y a en commun et estime précision, rappel et F1."""
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0, 0.0, 0.0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return precision, recall, f1

In [ ]:
def evaluation_generation(results): 
    # TODO
    return eval

In [ ]:
eval = evaluation_generation(results)
print(eval)

**Question :** Que pensez-vous de cette évaluation ? 

### 6.2 Analyse qualitative (à faire) 

Faites l'analyse des réponses de ce modèle. Présentez vos observations par rapport aux réponses obtenus des modèles des tâches 2 et 3. 

Expliquez ce que vous retenez des 3 dernières tâches sur le pré-entraînement et le post-entraînement du modèle GPT-2. 

Vous pouvez ajouter des cellules au besoin.